In [15]:
import pandas as pd
import numpy as np
import sys
sys.path.append(r'C:\Users\solba\dacon-project') 

from models.precessing import train_preprocess_data, test_preprocess_data

In [5]:
raw_train = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\train.csv")
raw_test = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\test.csv")

In [6]:
train_df = train_preprocess_data(raw_train)
test_df = test_preprocess_data(raw_test)

C:\Users\solba\dacon-project\models\precessing.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  personal['자격증보유개수'] = personal['certificate_acquisition'].apply(count_certificates)
C:\Users\solba\dacon-project\models\precessing.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  personal_df['job'] = le.fit_transform(personal_df['job'])
C:\Users\solba\dacon-project\models\precessing.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [7]:
train_df

,ID,학교,직업,재등록,자격증보유개수,수강개수,투자시간,수료여부
0,TRAIN_000,22,0,0,1,2,2,0
1,TRAIN_001,1,0,0,0,1,3,0
2,TRAIN_002,27,0,0,1,1,10,0
3,TRAIN_003,1,0,0,2,1,2,1
4,TRAIN_004,16,0,0,1,1,2,0
...,...,...,...,...,...,...,...,...
743,TRAIN_743,13,0,0,2,1,3,0
744,TRAIN_744,43,0,0,2,1,3,1
745,TRAIN_745,11,0,0,0,1,1,1
746,TRAIN_746,30,0,0,0,1,2,1


In [8]:
test_df

,ID,학교,직업,재등록,자격증보유개수,수강개수,투자시간
0,TEST_000,3,0,0,0,1,2
1,TEST_001,45,0,0,0,1,1
2,TEST_002,56,0,0,0,1,5
3,TEST_003,48,0,0,4,2,4
4,TEST_004,0,0,0,0,1,1
...,...,...,...,...,...,...,...
809,TEST_809,30,0,0,0,1,1
810,TEST_810,22,0,0,0,1,3
811,TEST_811,20,0,0,0,1,2
812,TEST_812,15,2,0,1,1,2


In [17]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# 1. 데이터 준비
X_train = train_df.drop(['ID', '수료여부'], axis=1)
y_train = train_df['수료여부']
X_test = test_df.drop(['ID'], axis=1)

# 2. 가중치 계산 (0의 개수 / 1의 개수)
zero_count = (y_train == 0).sum()
one_count = (y_train == 1).sum()
# 만약 1이 너무 적으면 가중치를 높게 줍니다.
weight_ratio = zero_count / one_count 

print(f"0의 개수: {zero_count}, 1의 개수: {one_count}")
print(f"적용 가중치: {weight_ratio:.2f}")

# 3. XGBoost 모델 생성 (불균형 최적화)
# scale_pos_weight가 0으로 쏠리는 현상을 막아줍니다.
xgb_model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    scale_pos_weight=weight_ratio, # 핵심 파라미터
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

# 4. 전체 데이터로 학습
xgb_model.fit(X_train, y_train)

# 5. 예측 및 분포 확인
test_predictions = xgb_model.predict(X_test)
unique, counts = np.unique(test_predictions, return_counts=True)
print("최종 예측 분포:", dict(zip(unique, counts)))

# 6. CSV 저장
submission = pd.DataFrame({'ID': test_df['ID'], '수료여부': test_predictions})
submission.to_csv('submission_xgb.csv', index=False)

0의 개수: 525, 1의 개수: 223
적용 가중치: 2.35
최종 예측 분포: {np.int64(0): np.int64(461), np.int64(1): np.int64(353)}


c:\Users\solba\anaconda3\envs\dacon\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:10:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [18]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

# 1. 데이터 로드 및 전처리
X_train = train_df.drop(['ID', '수료여부'], axis=1)
y_train = train_df['수료여부']
X_test = test_df.drop(['ID'], axis=1)

# 범주형으로 의심되는 컬럼들 (학교, 직업) 처리
# 이 컬럼들을 원-핫 인코딩하여 수치적 오해를 방지합니다.
categorical_cols = ['학교', '직업']
X_combined = pd.concat([X_train, X_test])
X_combined_encoded = pd.get_dummies(X_combined, columns=categorical_cols)

# 다시 train/test로 분리
X_train_final = X_combined_encoded[:len(train_df)]
X_test_final = X_combined_encoded[len(train_df):]

# 2. 모델 생성 (과적합 방지에 집중)
# 데이터가 적으므로 max_depth를 5~7 정도로 낮게 잡는 것이 핵심입니다.
rf_model = RandomForestClassifier(
    n_estimators=300, 
    max_depth=6, 
    min_samples_leaf=5,
    class_weight='balanced', # 0으로 쏠림 방지
    random_state=42
)

# 3. 학습 및 예측
rf_model.fit(X_train_final, y_train)
test_predictions = rf_model.predict(X_test_final)

# 4. 결과 확인 및 저장
print("예측 분포:", pd.Series(test_predictions).value_counts())
submission = pd.DataFrame({'ID': test_df['ID'], '수료여부': test_predictions})
submission.to_csv('submission_rf_final.csv', index=False)

예측 분포: 0    485
1    329
Name: count, dtype: int64


In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

# 1. 데이터 전처리
X_train = train_df.drop(['ID', '수료여부'], axis=1)
y_train = train_df['수료여부']
X_test = test_df.drop(['ID'], axis=1)

# 범주형 데이터 원-핫 인코딩
categorical_cols = ['학교', '직업']
X_combined = pd.concat([X_train, X_test])
X_combined_encoded = pd.get_dummies(X_combined, columns=categorical_cols)

X_train_final = X_combined_encoded[:len(train_df)]
X_test_final = X_combined_encoded[len(train_df):]

# 스케일링 (로지스틱 회귀를 위해 필요)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_final)
X_test_scaled = scaler.transform(X_test_final)

# 2. 개별 모델 정의
# 모델 1: Random Forest (안정성)
rf = RandomForestClassifier(n_estimators=300, max_depth=5, class_weight='balanced', random_state=42)

# 모델 2: XGBoost (패턴 포착)
xgb = XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=4, scale_pos_weight=2, random_state=42)

# 모델 3: Logistic Regression (선형 관계 보완)
lr = LogisticRegression(class_weight='balanced', random_state=42)

# 3. 보팅 앙상블 (Soft Voting)
# 각 모델이 낸 '확률'의 평균을 구해 최종 예측합니다.
voting_model = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('lr', lr)],
    voting='soft'
)

# 4. 학습 및 예측
voting_model.fit(X_train_scaled, y_train)
final_preds = voting_model.predict(X_test_scaled)

# 5. 결과 저장
print("예측 분포 확인:\n", pd.Series(final_preds).value_counts())
submission = pd.DataFrame({'ID': test_df['ID'], '수료여부': final_preds})
submission.to_csv('submission_ensemble.csv', index=False)

예측 분포 확인:
 0    491
1    323
Name: count, dtype: int64


In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler

# 1. 피처 엔지니어링 함수
def preprocess_data(df):
    # (1) 투자시간 구간화: 노이즈 제거
    df['투자시간_bin'] = pd.cut(df['투자시간'], bins=[-1, 2, 5, 100], labels=[0, 1, 2]).astype(int)
    
    # (2) 파생 변수: 활동 지수 (자격증 + 수강개수)
    df['활동지수'] = df['자격증보유개수'] + df['수강개수']
    
    # (3) 학교, 직업 원-핫 인코딩을 위해 범주형으로 변환
    df['학교'] = df['학교'].astype(str)
    df['직업'] = df['직업'].astype(str)
    return df

# 데이터 로드 및 전처리 적용
train = preprocess_data(train_df.copy())
test = preprocess_data(test_df.copy())

X_train = train.drop(['ID', '수료여부'], axis=1)
y_train = train['수료여부']
X_test = test.drop(['ID'], axis=1)

# 범주형 인코딩
X_combined = pd.concat([X_train, X_test])
X_combined_encoded = pd.get_dummies(X_combined)

X_train_final = X_combined_encoded[:len(train_df)]
X_test_final = X_combined_encoded[len(train_df):]

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_final)
X_test_scaled = scaler.transform(X_test_final)

# 2. 강력한 모델 3종 세트 (하이퍼파라미터 더 타이트하게 조정)
# RF: 깊이를 4로 더 줄여서 과적합 방지
rf = RandomForestClassifier(n_estimators=500, max_depth=4, class_weight='balanced', random_state=42)

# XGB: 학습률을 낮추고 더 꼼꼼하게 학습
xgb = XGBClassifier(n_estimators=300, learning_rate=0.005, max_depth=3, scale_pos_weight=2.5, random_state=42)

# LGBM: 작은 데이터셋에 강점 (파라미터 주의)
lgbm = LGBMClassifier(n_estimators=200, learning_rate=0.01, num_leaves=15, class_weight='balanced', random_state=42)

# 3. 앙상블 (Voting)
final_voting = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('lgbm', lgbm)],
    voting='soft'
)

# 4. 학습 및 결과 도출
final_voting.fit(X_train_scaled, y_train)
test_preds = final_voting.predict(X_test_scaled)

# 5. 저장
print("예측 분포:\n", pd.Series(test_preds).value_counts())
submission = pd.DataFrame({'ID': test_df['ID'], '수료여부': test_preds})
submission.to_csv('submission_ultimate.csv', index=False)

[LightGBM] [Info] Number of positive: 223, number of negative: 525
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001199 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 85
[LightGBM] [Info] Number of data points in the train set: 748, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
예측 분포:
 1    418
0    396
Name: count, dtype: int64


c:\Users\solba\anaconda3\envs\dacon\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [21]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler

# 1. 데이터 고도화 전처리
def super_preprocess(df):
    # (1) 시너지 피처: 투자시간 * 활동지수
    df['활동지수'] = df['자격증보유개수'] + df['수강개수']
    df['집중도'] = df['투자시간'] * df['활동지수']
    
    # (2) 투자시간 상위 그룹 여부 (편향 방지)
    df['열혈수강생'] = (df['투자시간'] >= 5).astype(int)
    
    # (3) 학교, 직업 문자열 변환
    df['학교'] = 'SCH_' + df['학교'].astype(str)
    df['직업'] = 'JOB_' + df['직업'].astype(str)
    return df

train = super_preprocess(train_df.copy())
test = super_preprocess(test_df.copy())

X_train = train.drop(['ID', '수료여부'], axis=1)
y_train = train['수료여부']
X_test = test.drop(['ID'], axis=1)

# 범주형 인코딩 (get_dummies)
X_combined = pd.concat([X_train, X_test])
X_combined_encoded = pd.get_dummies(X_combined)

X_train_final = X_combined_encoded[:len(train_df)]
X_test_final = X_combined_encoded[len(train_df):]

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_final)
X_test_scaled = scaler.transform(X_test_final)

# 2. 정예 모델 앙상블 (파라미터 미세 튜닝)
# 과적합을 막기 위해 나무 숫자는 늘리고 깊이는 극단적으로 낮춤
rf = RandomForestClassifier(n_estimators=1000, max_depth=4, min_samples_leaf=10, class_weight='balanced', random_state=42)
xgb = XGBClassifier(n_estimators=500, learning_rate=0.005, max_depth=3, subsample=0.8, scale_pos_weight=2.8, random_state=42)
lgbm = LGBMClassifier(n_estimators=500, learning_rate=0.005, num_leaves=10, subsample=0.7, class_weight='balanced', random_state=42)

# 3. 보팅 앙상블
voting_model = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb), ('lgbm', lgbm)],
    voting='soft'
)

# 4. 학습 및 확률 예측
voting_model.fit(X_train_scaled, y_train)
# 클래스 1(수료)에 대한 확률값 추출
proba_1 = voting_model.predict_proba(X_test_scaled)[:, 1]

# 5. 임계값 조정 (가장 중요!)
# 보통 0.5를 기준으로 0, 1을 나누지만, 점수가 낮을 땐 0.4~0.45 사이가 유리할 수 있습니다.
threshold = 0.43 
final_preds = (proba_1 >= threshold).astype(int)

# 6. 결과 저장
print(f"현재 임계값: {threshold}")
print("예측 분포:\n", pd.Series(final_preds).value_counts())
submission = pd.DataFrame({'ID': test_df['ID'], '수료여부': final_preds})
submission.to_csv('submission_pro.csv', index=False)

[LightGBM] [Info] Number of positive: 223, number of negative: 525
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000168 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 102
[LightGBM] [Info] Number of data points in the train set: 748, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
현재 임계값: 0.43
예측 분포:
 1    722
0     92
Name: count, dtype: int64


c:\Users\solba\anaconda3\envs\dacon\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier # 추가 설치 필요: pip install catboost
from sklearn.preprocessing import StandardScaler

# 1. 고도화된 전처리 (이전보다 더 단순화하여 일반화 성능 높임)
def final_preprocess(df):
    # 활동성 지수: 노이즈를 줄이기 위해 단순히 합산
    df['활동성'] = df['자격증보유개수'] + df['수강개수']
    # 투자시간의 중요도를 강조하되, 이상치 영향 최소화
    df['효율성'] = df['활동성'] / (df['투자시간'] + 1) 
    
    # 범주형은 그대로 문자열 처리
    df['학교'] = df['학교'].astype(str)
    df['직업'] = df['직업'].astype(str)
    return df

train = final_preprocess(train_df.copy())
test = final_preprocess(test_df.copy())

X_train = train.drop(['ID', '수료여부'], axis=1)
y_train = train['수료여부']
X_test = test.drop(['ID'], axis=1)

# 원-핫 인코딩
X_combined_encoded = pd.get_dummies(pd.concat([X_train, X_test]))
X_train_final = X_combined_encoded[:len(train_df)]
X_test_final = X_combined_encoded[len(train_df):]

# 2. 개별 정예 모델 정의 (가장 안정적인 파라미터 조합)
rf = RandomForestClassifier(n_estimators=1000, max_depth=5, class_weight='balanced', random_state=42)

# CatBoost: 범주형 데이터의 왕, 과적합에 매우 강함
cat = CatBoostClassifier(iterations=500, learning_rate=0.01, depth=4, l2_leaf_reg=5, 
                         verbose=0, random_state=42, auto_class_weights='Balanced')

xgb = XGBClassifier(n_estimators=500, learning_rate=0.005, max_depth=3, scale_pos_weight=2.5, random_state=42)

# 3. 가중치 보팅 (Weighted Voting)
# 성능이 더 안정적인 RF와 CatBoost에 더 높은 가중치를 부여 (2:2:1 비율)
voting_model = VotingClassifier(
    estimators=[('rf', rf), ('cat', cat), ('xgb', xgb)],
    voting='soft',
    weights=[2, 2, 1] 
)

# 4. 학습 및 예측
voting_model.fit(X_train_final, y_train)
proba_1 = voting_model.predict_proba(X_test_final)[:, 1]

# 5. 임계값(Threshold) 미세 조정
# 0.39에서 떨어졌다면, 이번엔 1을 조금 더 보수적으로 잡아서(0.48) 정확도를 높여봅니다.
threshold = 0.48 
final_preds = (proba_1 >= threshold).astype(int)

# 6. 결과 저장
print(f"예측된 수료자(1) 수: {sum(final_preds)}")
submission = pd.DataFrame({'ID': test_df['ID'], '수료여부': final_preds})
submission.to_csv('submission_final_boss.csv', index=False)